In [3]:
from creds import HOST_NAME, DB_NAME, USERNAME , PASSWORD 
import pandas as pd
import psycopg
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def exec_query_os(query):
    conn = psycopg.connect(
        dbname=DB_NAME, user=USERNAME, password=PASSWORD, host=HOST_NAME, port="5432"
    )
    df = pd.read_sql(query, conn)

    conn.close()
    return df

def df_to_parquet(df, table_name):
    df.to_parquet(f"./output/{table_name}.parquet", compression='gzip')


## Reading NBA Data from db

In [ ]:
game_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_GAME_BOX_HIST"
                where season = 2024;
            """)

game_pbp_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PBP_HIST"
                where season = 2024;
            """)

player_box_df = exec_query_os("""
                SELECT *
                FROM public."NBA_PLAYER_BOX_HIST"
                where season = 2024;
            """)


In [5]:
print(game_box_df.shape)
print(game_pbp_df.shape)
print(player_box_df.shape)


(2628, 57)
(611682, 64)
(34867, 57)
